In [30]:
import pandas as pd
import MetaTrader5 as mt5
import talib as ta
from datetime import datetime,timedelta
from out_in_bb_backtest import setup_tsl,setup_stop_tp,setup_estr_2
import numpy as np
from backtesting import Strategy, Backtest
import warnings
warnings.filterwarnings("ignore")

if not mt5.initialize():
    print("initialize() failed")
    mt5.shutdown()

#ativo = 'EURUSD'
ativo = 'WDO@N'
df = pd.DataFrame(mt5.copy_rates_from_pos(ativo, mt5.TIMEFRAME_M1, 0, 40000))
df['Data'] = pd.to_datetime(df['time'], unit='s').apply(lambda x: str(x))
df = df[['Data','open','high','low','close','real_volume']]
df.columns = ['Data','Open','High','Low','Close','Vol']
df_puro = df.rename(columns = {'real_volume':'vol'})
df2 = df_puro.copy()

In [31]:
df2

,Data,Open,High,Low,Close,Vol
0,2022-10-10 09:39:00,5210.0,5212.0,5209.5,5211.5,4187
1,2022-10-10 09:40:00,5211.0,5214.5,5211.0,5211.5,7115
2,2022-10-10 09:41:00,5211.5,5212.5,5210.0,5210.0,5252
3,2022-10-10 09:42:00,5210.0,5211.0,5209.0,5210.0,7384
4,2022-10-10 09:43:00,5210.0,5210.0,5202.0,5204.5,30656
...,...,...,...,...,...,...
39995,2023-01-23 10:54:00,5218.0,5219.5,5216.0,5216.0,8434
39996,2023-01-23 10:55:00,5216.0,5217.0,5213.0,5213.0,6174
39997,2023-01-23 10:56:00,5213.0,5216.5,5213.0,5213.0,6439
39998,2023-01-23 10:57:00,5213.0,5214.0,5210.0,5212.0,12701


In [34]:
time_period = 14
var_bb = 1.8
pontos = 10
rate_stop = 1
rate_tp = 1
spread = 0.5
timeframe = 5

df=df_puro.copy()
bt = setup_estr_2(df,timeperiod=time_period,var_bb=var_bb,pontos = pontos,rate_tp = rate_tp,EMA=False)
#bt = setup_tsl(df,timeperiod=time_period,var_bb=var_bb,pontos = pontos,rate_tp = rate_tp,EMA=False)

stats = bt.run()
#bt.plot()

df_trades = stats['_trades']
#df_trades = df_trades.drop_duplicates('EntryTime')
df_trades['Hora_h'] = df_trades['EntryTime'].apply(lambda x: x.hour)
df_trades['resultado_binario'] = df_trades['ReturnPct'].apply(lambda x: 1 if x > 0 else 0)

print(f'Time_period: {time_period} -- var_bb: {var_bb} -- pontos: {pontos}--Rate_tp: {rate_tp}')
df_new_hora = df_trades[['Hora_h','resultado_binario']]
df_new_hora = df_trades.groupby(['Hora_h'],as_index=False).agg(qtd_operacoes=('resultado_binario', 'count'), taxa_acerto=('resultado_binario', 'mean'))

#df_new_hora['pontos_liquido'] = df_new_hora.apply(calc_pontos, axis = 1)
print(df_new_hora)

Time_period: 14 -- var_bb: 1.8 -- pontos: 10--Rate_tp: 1
   Hora_h  qtd_operacoes  taxa_acerto
0       9              1     1.000000
1      10              9     0.555556
2      11              6     0.833333
3      12              3     0.666667
4      13              3     1.000000
5      14              6     0.833333
6      15              4     0.500000
7      16              2     0.500000
8      17              4     0.750000
9      18              2     0.000000


In [35]:
df_new_hora['qtd_operacoes'].sum()

40

In [29]:
result = pd.read_excel('resultado_win_1M_ema_True_stop_tp.xlsx')
result.sort_values(by = 'Taxa_acerto', ascending=False).iloc[0:30]

FileNotFoundError: [Errno 2] No such file or directory: 'resultado_win_1M_ema_True_stop_tp.xlsx'

In [24]:
for index,result_atual in result.loc[[104,68,4,12,20]].iterrows():
    time_period = result_atual['Time_period']
    var_bb = result_atual['var_bb']
    pontos = result_atual['pontos']
    rate_stop = 1
    rate_tp = result_atual['Rate_tp']
    retorno = result_atual['Retorno']
    Pontos_total = result_atual['Pontos_total']
    spread = 5
    timeframe = 5


    df=df_puro.copy()
    bt = setup_stop_tp(df,timeperiod=time_period,var_bb=var_bb,pontos = pontos,rate_tp = rate_tp,EMA=True)
    #bt = setup_tsl(df,timeperiod=time_period,var_bb=var_bb,pontos = pontos,rate_tp = rate_tp,EMA=False)

    stats = bt.run()

    df_trades = stats['_trades']
    #df_trades = df_trades.drop_duplicates('EntryTime')
    df_trades['Hora_h'] = df_trades['EntryTime'].apply(lambda x: x.hour)
    df_trades['resultado_binario'] = df_trades['ReturnPct'].apply(lambda x: 1 if x > 0 else 0)

    print(f'Time_period: {time_period} -- var_bb: {var_bb} -- pontos: {pontos}--Rate_tp: {rate_tp}-- Retorno: {round(retorno,2)} -- Pontos_total: {Pontos_total}')
    df_new_hora = df_trades[['Hora_h','resultado_binario']]
    df_new_hora = df_trades.groupby(['Hora_h'],as_index=False).agg(qtd_operacoes=('resultado_binario', 'count'), taxa_acerto=('resultado_binario', 'mean'))

    #df_new_hora['pontos_liquido'] = df_new_hora.apply(calc_pontos, axis = 1)
    print(df_new_hora)


Time_period: 30.0 -- var_bb: 2.2 -- pontos: 150.0--Rate_tp: 1.0-- Retorno: 2.32 -- Pontos_total: 2999.9944
   Hora_h  qtd_operacoes  taxa_acerto
0       9             30     0.700000
1      10             47     0.531915
2      11             39     0.435897
3      12             33     0.515152
4      13             39     0.512821
5      14             39     0.589744
6      15             30     0.566667
7      16             34     0.647059
8      17             32     0.406250
9      18              4     0.500000
Time_period: 20.0 -- var_bb: 2.2 -- pontos: 200.0--Rate_tp: 1.0-- Retorno: 4.62 -- Pontos_total: 3599.9912
   Hora_h  qtd_operacoes  taxa_acerto
0       9             51     0.607843
1      10             62     0.403226
2      11             49     0.571429
3      12             56     0.517857
4      13             48     0.604167
5      14             51     0.607843
6      15             53     0.452830
7      16             50     0.520000
8      17             52  

In [ ]:
new[new['signal'] != 0][['signal']].tail(10)

In [ ]:
stats['_trades'].tail(30)

#### Setup_Tsl

In [ ]:
result = pd.read_excel('resultado_win_1M_ema_False_tls.xlsx')
result.sort_values(by = 'Retorno', ascending=False).iloc[:]

In [ ]:
for index,result_atual in result.loc[[0,7,6]].iterrows():
    time_period = result_atual['Time_period']
    var_bb = result_atual['var_bb']
    pontos = result_atual['pontos']
    rate_stop = 1
    #rate_tp = result_atual['Rate_tp']
    retorno = result_atual['Retorno']
    #Pontos_total = result_atual['Pontos_total']
    spread = 0.00005
    timeframe = 5


    df=df_puro.copy()
    bt = setup_tsl(df,timeperiod=time_period,var_bb=var_bb,pontos = pontos,EMA=True)

    stats = bt.run()

    df_trades = stats['_trades']
    df_trades = df_trades.drop_duplicates('EntryTime')
    df_trades['Hora_h'] = df_trades['EntryTime'].apply(lambda x: x.hour)
    df_trades['resultado_binario'] = df_trades['ReturnPct'].apply(lambda x: 1 if x > 0 else 0)

    print(f'Time_period: {time_period} -- var_bb: {var_bb} -- pontos: {pontos}-- Retorno: {round(retorno,2)}')
    df_new_hora = df_trades[['Hora_h','resultado_binario']]
    df_new_hora = df_trades.groupby(['Hora_h'],as_index=False).agg(qtd_operacoes=('resultado_binario', 'count'), taxa_acerto=('resultado_binario', 'mean'))

    #df_new_hora['pontos_liquido'] = df_new_hora.apply(calc_pontos, axis = 1)
    print(df_new_hora)



In [ ]:
bt.plot()

In [ ]:
df_trades = stats['_trades']
df_trades['Hora_h'] = df_trades['EntryTime'].apply(lambda x: x.hour)
df_trades['resultado_binario'] = df_trades['ReturnPct'].apply(lambda x: 1 if x > 0 else 0)

print(f'Time_period: {time_period} -- var_bb: {var_bb} -- pontos: {pontos}-- Retorno: {round(retorno,2)} ')
df_new_hora = df_trades[['Hora_h','resultado_binario']]
df_new_hora = df_trades.groupby(['Hora_h'],as_index=False).agg(qtd_operacoes=('resultado_binario', 'count'), taxa_acerto=('resultado_binario', 'mean'))

#df_new_hora['pontos_liquido'] = df_new_hora.apply(calc_pontos, axis = 1)
print(df_new_hora)